<a href="https://colab.research.google.com/github/leoxiang66/Knowledge-driven-spoken-dialogue/blob/main/%E8%AE%AD%E7%BB%83%E5%AF%B9%E8%AF%9D%E6%A8%A1%E5%9E%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 14.5 MB/s 
     |████████████████████████████████| 365 kB 73.2 MB/s 
     |████████████████████████████████| 6.6 MB 63.4 MB/s 
     |████████████████████████████████| 101 kB 14.5 MB/s 
     |████████████████████████████████| 212 kB 63.0 MB/s 
     |████████████████████████████████| 115 kB 94.4 MB/s 
     |████████████████████████████████| 127 kB 51.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import transformers, datasets
print(transformers.__version__)
print(datasets.__version__)

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("uer/bart-base-chinese-cluecorpussmall")

model = AutoModelForSeq2SeqLM.from_pretrained("uer/bart-base-chinese-cluecorpussmall")

In [ ]:
tokenizer.model_max_length = 1024
tokenizer.model_max_length

In [ ]:
special_token_dict = {'additional_special_tokens': ["[ett]", '[usr]','[sys]','[klg]']}
num_tokens = tokenizer.add_special_tokens(special_token_dict)

In [ ]:
num_tokens

In [ ]:
model.resize_token_embeddings(len(tokenizer))

In [ ]:
tokenizer('hello', 'have',truncation='only_second')

In [ ]:
from datasets import load_dataset

dataset = load_dataset("Adapting/Knowledge-Driven-Dialogues")

In [ ]:
dataset

In [ ]:
import ast
def preprocess_function(example):
    chat_history = example['chat_history']
    chat_history_list = ast.literal_eval(chat_history)
    chat_history = ''
    for i in range(0,len(chat_history_list)-1,2):
        chat_history += '[usr] '
        chat_history += chat_history_list[i]
        chat_history += ' [sys] '
        chat_history += chat_history_list[i+1]
        chat_history += ' '

    chat_history += f'[usr] {chat_history_list[-1]}'
    knowledge = example['knowledge'] 
    if knowledge is None: 
        knowledge = '[klg]'
    else:
        knowledge = f'[klg] {knowledge}'

    
    # print(chat_history)
    input = knowledge +' '+chat_history

    model_inputs = tokenizer(input, truncation=True)

    response = example['response']
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(response, truncation=True)
    
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [ ]:
preprocess_function(dataset['train'][2])

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, remove_columns=dataset['train'].column_names)

In [ ]:
tokenized_datasets

In [ ]:
len(max(tokenized_datasets['train']['input_ids']))

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import torch
GPU = torch.cuda.is_available()
torch.cuda.empty_cache()

from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    "knowledge_driven_dialogue",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    # predict_with_generate=True, 
    fp16=GPU,# speeds up training on modern GPUs.
    eval_accumulation_steps = 10,
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,

)

In [ ]:
trainer.train()

In [ ]:
# from huggingface_hub import notebook_login
# notebook_login()